In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [2]:
wisconsin = pd.read_csv('C:/Users/dbda.STUDENTSDC/Desktop/M.L/Cases/Wisconsin/BreastCancer.csv', index_col = 0)
wisconsin.head()

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,Benign
63375,9,1,2,6,4,10,7,7,2,Malignant
76389,10,4,7,2,2,8,6,1,1,Malignant
95719,6,10,10,10,8,10,7,10,7,Malignant
128059,1,1,1,1,2,5,5,1,1,Benign


In [3]:
X,y = wisconsin.drop('Class',axis=1),wisconsin['Class']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

Without scaling

In [4]:
svm = SVC(kernel='linear')
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.9714285714285714


With Scaling

In [5]:
scaler = StandardScaler()
pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9666666666666667


With MinMax Scaling

In [6]:
scaler = MinMaxScaler()
pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9714285714285714


using all three scales at one

In [7]:
scales = [StandardScaler(), MinMaxScaler(), None]

for scaler in scales:
    pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    print(f'{scaler}: {accuracy_score(y_test, y_pred)}')

StandardScaler(): 0.9666666666666667
MinMaxScaler(): 0.9714285714285714
None: 0.9714285714285714


In [8]:
hr = pd.read_csv('C:/Users/dbda.STUDENTSDC/Desktop/M.L/Datasets/HR_comma_sep.csv')
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [ ]:
X,y = hr.drop('left',axis=1),hr['left']
ohe = OneHotEncoder(drop='first',sparse_output=False).set_output(transform='pandas')

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

from sklearn.compose import ColumnTransformer, make_column_selector

col_trnf = ColumnTransformer([('OHE', ohe, make_column_selector(dtype_include=object))],
                              remainder='passthrough', verbose_feature_names_out=False)

In [11]:
ss = StandardScaler()
pipe = Pipeline([('TRNSF', col_trnf), ('SS', ss), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
accuracy_score(y_test, y_pred)

0.7819515447877307

In [14]:
Cs = np.linspace(0.01,5,15)
scores = []
for c in Cs:
    svm = SVC(kernel='linear',C=c)
    pipe = Pipeline([('TRNSF',col_trnf),("SCL",ss),('SVM',svm)])
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    scores.append([c,accuracy_score(y_test,y_pred)])
df_score = pd.DataFrame(scores,columns=['C','score']) 
df_score.sort_values('score',ascending=False)


,C,score
0,0.010000,0.786397
1,0.366429,0.781952
2,0.722857,0.781952
3,1.079286,0.781952
4,1.435714,0.781952
5,1.792143,0.781952
6,2.148571,0.781952
7,2.505000,0.781952
8,2.861429,0.781952
9,3.217857,0.781952


Polynomial kernel

In [ ]:
svm = SVC(kernel='poly')
pipe = Pipeline([('TRNSF', col_trnf), ('SS', ss), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
accuracy_score(y_test, y_pred)


0.9415425650144477

Radial kernel

In [17]:
svm = SVC(kernel='rbf')
pipe = Pipeline([('TRNSF', col_trnf), ('SS', ss), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
accuracy_score(y_test, y_pred)

0.9524338741942654

In [ ]:
# For Different Cs and different degrees in Poly SVM
deg = [2,3,4]
score = []

for c in tqdm(np.linspace(0.0001, 5, 15)):
    for d in deg:
        svm = SVC(kernel='poly', C=c, degree = d)
        pipe = Pipeline([('TRNSF', col_trnf), ('SS', ss), ('SVM', svm)])
        pipe.fit(x_train,y_train)
        y_pred = pipe.predict(x_test)
        # print('For C = ', c)
        # print('For degree = ', d)
        # print('Accuracy Score: ', accuracy_score(y_test, y_pred))
        # print('\n')
        score.append([c,d,accuracy_score(y_test, y_pred)])

df_score = pd.DataFrame(score, columns = ['C', 'Degree', 'Accuracy Score'])
df_score.sort_values('Accuracy Score', ascending=False)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [01:18<00:00,  5.21s/it]


,C,Degree,Accuracy Score
35,3.928593,4,0.956879
32,3.571457,4,0.956657
29,3.214321,4,0.956435
38,4.285729,4,0.956212
44,5.000000,4,0.955990
26,2.857186,4,0.955768
41,4.642864,4,0.955768
23,2.500050,4,0.955101
20,2.142914,4,0.953990
17,1.785779,4,0.953545


In [24]:
# For Different Cs and different Gammas in RBF SVM
score = []

for c in tqdm(np.linspace(0.01, 5, 10)):
    for g in np.linspace(0.01, 5, 10):
        svm = SVC(kernel='rbf', C=c, gamma=g)
        pipe = Pipeline([('TRNSF', col_trnf), ('SS', ss), ('SVM', svm)])
        pipe.fit(x_train,y_train)
        y_pred = pipe.predict(x_test)
        # print('For C = ', c)
        # print('For degree = ', d)
        # print('Accuracy Score: ', accuracy_score(y_test, y_pred))
        # print('\n')
        score.append([c,g,accuracy_score(y_test, y_pred)])

df_score = pd.DataFrame(score, columns = ['C', 'Gamma', 'Accuracy Score'])
df_score.sort_values('Accuracy Score', ascending=False)

100%|██████████| 10/10 [18:20<00:00, 110.08s/it]


,C,Gamma,Accuracy Score
32,1.673333,1.118889,0.982885
83,4.445556,1.673333,0.981107
63,3.336667,1.673333,0.981107
42,2.227778,1.118889,0.981107
73,3.891111,1.673333,0.981107
...,...,...,...
6,0.010000,3.336667,0.777062
7,0.010000,3.891111,0.773950
8,0.010000,4.445556,0.771060
9,0.010000,5.000000,0.769060
